In [2]:
using Plots, Distributions

In [23]:
# Simulates a single realization of the markov chain for 60 days
function simulate_realization(M, s1, s2)
    # Number of days
    days = 60
    
    # State
    # number of worn machines
    W = zeros(Int, days)
    # number of broken machines
    B = zeros(Int, days)
    
    # probability of new -> worn
    p12 = (1/60) + (1/200 - 1/60) * sqrt(1 - ((10 - s1) / 10)^2)
    # probability of worn -> broken
    p23 = (1/30) + (1/100 - 1/30) * sqrt(1 - ((10 - s2) / 10)^2)
    
    # Simulate all days
    for i in 1:(days-1)
        B12 = rand(Binomial(M-W[i]-B[i], p12))
        B23 = rand(Binomial(W[i], p23))
        
        W[i+1] = W[i] + B12 - B23
        B[i+1] = B[i] +     + B23 - B[i]
    end
    
    # Calculate the number of new machines every day
    N = M * ones(days) - W - B
    
    N, W, B
end

# Performs a simulation experiment that calculates the gain at day 60
function simulate(M, s1, s2)
    gain = 0.0
    
    realizations = 10_000
    for _ in 1:realizations
        N, W, B = simulate_realization(M, s1, s2)
        gain += 1 * (N[end] + W[end]) - 90 * B[end] - s1 - s2
    end
    
    return gain / realizations
end

# Parameters
# Total number of machines
M = 2_000
# money spent daily for maintainance of worn machines
s1 = 4
# money spent daily for maintainance of broken machines
s2 = 6
simulate(M, s1, s2)

1429.895

In [5]:
X = []
y = []
# Calculate the 60th-day gain for all pairs of even numbers from 2 to 10
@time for j=2:2:10, k=2:2:10
    gain = simulate(M, j, k)
    push!(X, [j, k])
    push!(y, [gain])
    println("($j, $k) = $(gain)")
end

(2, 2) = 1039.3989
(2, 4) = 1188.3406
(2, 6) = 1290.7722
(2, 8) = 1359.9979
(2, 10) = 1376.7439
(4, 2) = 1228.1076
(4, 4) = 1345.6725
(4, 6) = 1425.6817
(4, 8) = 1481.0663
(4, 10) = 1494.6819
(6, 2) = 1348.1477
(6, 4) = 1447.8584
(6, 6) = 1520.7878
(6, 8) = 1559.7378
(6, 10) = 1567.3565
(8, 2) = 1415.8446
(8, 4) = 1505.7182
(8, 6) = 1566.1442
(8, 8) = 1602.6372
(8, 10) = 1618.0364
(10, 2) = 1440.5986
(10, 4) = 1523.447
(10, 6) = 1578.8589
(10, 8) = 1613.3044
(10, 10) = 1629.286
  6.530749 seconds (1.56 M allocations: 827.385 MiB, 1.46% gc time)


In [7]:
using Random

sigma(x) = 1 / (1 + exp(-x))
sigma_prime(x) = sigma(x) * (1 - sigma(x))

function nn(x, W1, b1, W2, b2)
    z1 = W1 * x + b1
    a1 = sigma.(z1)
    z2 = W2 * a1 + b2
    a2 = z2
    return a2[1]
end

function compute_loss(W1, b1, W2, b2)
    loss = 0.0
    for (x, y_true) in zip(X, y)
        y_hat = nn(x, W1, b1, W2, b2)

        loss += (y_hat[1] - y_true[1])^2
    end
    loss /= (2 * size(X, 1))
    return loss
end
Random.seed!(1234);

mu = 1e-4

INPUT_SIZE = 2
HIDDEN_LAYER_SIZE = 100
W1 = randn(HIDDEN_LAYER_SIZE, INPUT_SIZE)
b1 = randn(HIDDEN_LAYER_SIZE, 1)

OUTPUT_LAYER_SIZE = 1
W2 = randn(OUTPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE)
b2 = randn(OUTPUT_LAYER_SIZE, 1)

for it in 1:1_000
    for (x, y_true) in zip(X, y)
        # Forward step to evaluate intermediate results
        z1 = W1 * x + b1
        a1 = sigma.(z1)
        z2 = W2 * a1 + b2
        a2 = z2

        # Backward step to evaluate parameter derivatives w.r.t. loss
        # d(loss)/d(z2) is `dz2`
        dz2 = a2 - y_true
        dW2 = dz2 * a1'
        db2 = dz2

        dz1 = (W2' * dz2) .* sigma_prime.(z1)
        dW1 = dz1 * x'
        db1 = dz1

        # Steepest descent to find optimal parameters
        W2 -= mu * dW2
        b2 -= mu * db2

        W1 -= mu * dW1
        b1 -= mu * db1
    end
    
    mu *= 0.9995
    
    if it % 100 == 0
        println("[$it]: $(round(compute_loss(W1, b1, W2, b2), digits=3))")
    end
end

[100]: 170.092
[200]: 36.246
[300]: 15.659
[400]: 7.213
[500]: 3.551
[600]: 2.806
[700]: 2.322
[800]: 1.921
[900]: 1.675
[1000]: 1.516


In [15]:
for x in [[0.2, 0.5], [0.3, 0.9], [0.1, 0.9]]
    gain_nn = nn(x, W1, b1, W2, b2)
    gain_simulation = simulate(M, x[1], x[2])
    println("Gain NN: $(round(gain_nn, digits=3)), Gain Simulation: $(round(gain_simulation, digits=3))")
end

Gain NN: 664.736, Gain Simulation: 483.158
Gain NN: 726.547, Gain Simulation: 592.14
Gain NN: 694.046, Gain Simulation: 503.288


In [1]:
function maximize_gain()
    x = [rand(2:2:10), rand(2:2:10)]
    
    mu = 1
    
    C = 0.1
    
    for m in 1:1000
        h = rand([1, -1], size(x, 1))
        c = C / m^0.2
        
        if m % 100 == 0 
            println("$(round.(x, digits=2)), $(round(nn(x, W1, b1, W2, b2), digits=0))")
            println("c=$(round(c, digits=3))")
        end
        
        finite_diff = nn(x + c*h, W1, b1, W2, b2) - nn(x - c*h, W1, b1, W2, b2)
        dx = [finite_diff / 2*c*h[i] for i in 1:size(x, 1)]
        
        x += mu * dx
    end
end

maximize_gain()

LoadError: UndefVarError: nn not defined